In [1]:
import pandas as pd 
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import string

from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import operator

In [2]:

data = pd.read_csv('C:/Users/Rhea/Documents/INNOVATION/PROJECT/Amazon_Phone_iPhone_32_Reviews.csv',
                   names=['Phone_Name','Date','Rating_Out_of_5','Review_Title','Review_Text'])



In [3]:
data.head()

,Phone_Name,Date,Rating_Out_of_5,Review_Title,Review_Text
0,"Apple iPhone 6 Unlocked Smartphone, 16 GB (Gold)",19-Apr-17,3,The phone is as good as new without any scratches,The phone is as good as new without any scratc...
1,"Apple iPhone 6 Unlocked Smartphone, 16 GB (Gold)",1-Jul-17,5,and great price. why buy a phone somewhere els...,purchased for my father. got here one day late...
2,"Apple iPhone 6 Unlocked Smartphone, 16 GB (Gold)",27-May-17,5,It's now been about 6 weeks since my purchase ...,"So I wanted to wait a while before reviewing, ..."
3,"Apple iPhone 6 Unlocked Smartphone, 16 GB (Gold)",28-Jan-17,5,Great phone,Phone came on time and was in very good condit...
4,"Apple iPhone 6 Unlocked Smartphone, 16 GB (Gold)",12-Jul-17,1,Camera sound won't turn off!!!,This is the second time we have attempted to o...


In [4]:
data.shape

(570, 5)

In [5]:
data['Review_Text'].values[5]

'It works, and it\'s clean. Professional packaging and it arrived fast.Downside, the battery must have not been replaced because it won\'t even last half a day idle. Good thing is that at least this isn\'t a waterproof phone and I can easily buy a battery replacement kit for less than $30 on Amazon. It really would\'ve been great if they put new battery so it won\'t suffer on performance due to shit iOS\'s "get a new phone or your old phone will be slow" tactic aka BATTERY GATE. I figured I would run into a problem like this since it\'s a refurbished one anyway. It\'s impossible to know the performance and battery life without activating the phone. I bet a lot of people have this problem but who would actually return it once it\'s activated? But hey, I\'m not paying a thousand for a new phone and I bought an iPhone knowing there\'s an ongoing battery gate controversy so I\'ll just suck it up and won\'t even bother sending a complaint.But, I chose to give it 3 stars since the price wasn

In [6]:
data_samples=data['Review_Text'].tolist()
data_samples[1]

'purchased for my father. got here one day late. no big deal. phone was unlocked. item came well packaged with all listed items. plus one glass screen protector and one plastic screen protector. not sure if it was a amazon freebie or seller freebie. was able to set up phone no problem. and great price. why buy a phone somewhere else and risk it. I will definitely buy another phone from amazon without a second thought.'

In [ ]:
data_samples = [w.lower() for w in data_samples]

In [7]:
type(data_samples)

list

### Processing Raw Text

For our language processing, we want to break up the string into words and punctuation. This step is called tokenization, and it produces our familiar structure, a list of words and punctuation.

In [9]:
tokenized_docs = [word_tokenize(doc) for doc in data_samples]
print(tokenized_docs[5])

['It', 'works', ',', 'and', 'it', "'s", 'clean', '.', 'Professional', 'packaging', 'and', 'it', 'arrived', 'fast.Downside', ',', 'the', 'battery', 'must', 'have', 'not', 'been', 'replaced', 'because', 'it', 'wo', "n't", 'even', 'last', 'half', 'a', 'day', 'idle', '.', 'Good', 'thing', 'is', 'that', 'at', 'least', 'this', 'is', "n't", 'a', 'waterproof', 'phone', 'and', 'I', 'can', 'easily', 'buy', 'a', 'battery', 'replacement', 'kit', 'for', 'less', 'than', '$', '30', 'on', 'Amazon', '.', 'It', 'really', 'would', "'ve", 'been', 'great', 'if', 'they', 'put', 'new', 'battery', 'so', 'it', 'wo', "n't", 'suffer', 'on', 'performance', 'due', 'to', 'shit', 'iOS', "'s", '``', 'get', 'a', 'new', 'phone', 'or', 'your', 'old', 'phone', 'will', 'be', 'slow', "''", 'tactic', 'aka', 'BATTERY', 'GATE', '.', 'I', 'figured', 'I', 'would', 'run', 'into', 'a', 'problem', 'like', 'this', 'since', 'it', "'s", 'a', 'refurbished', 'one', 'anyway', '.', 'It', "'s", 'impossible', 'to', 'know', 'the', 'performa

## Removing punctuation
Punctuation can help with tokenizers, but once you've done that, there's no reason to keep it around. There are tons of ways to remove punctuation. Since we have already learned regex, how would we do this?

In [10]:
print(string.punctuation)
str1=string.punctuation
str1=str1.replace('/','')
print(str1)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
!"#$%&'()*+,-.:;<=>?@[\]^_`{|}~


In [11]:
regex = re.compile('[%s]' % re.escape(str1)) #see documentation here: http://docs.python.org/2/library/string.html

tokenized_docs_no_punctuation = []

for review in tokenized_docs:
    
    new_review = []
    for token in review: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            new_review.append(new_token)
    
    tokenized_docs_no_punctuation.append(new_review)
    
print (tokenized_docs_no_punctuation[5])

['It', 'works', 'and', 'it', 's', 'clean', 'Professional', 'packaging', 'and', 'it', 'arrived', 'fastDownside', 'the', 'battery', 'must', 'have', 'not', 'been', 'replaced', 'because', 'it', 'wo', 'nt', 'even', 'last', 'half', 'a', 'day', 'idle', 'Good', 'thing', 'is', 'that', 'at', 'least', 'this', 'is', 'nt', 'a', 'waterproof', 'phone', 'and', 'I', 'can', 'easily', 'buy', 'a', 'battery', 'replacement', 'kit', 'for', 'less', 'than', '30', 'on', 'Amazon', 'It', 'really', 'would', 've', 'been', 'great', 'if', 'they', 'put', 'new', 'battery', 'so', 'it', 'wo', 'nt', 'suffer', 'on', 'performance', 'due', 'to', 'shit', 'iOS', 's', 'get', 'a', 'new', 'phone', 'or', 'your', 'old', 'phone', 'will', 'be', 'slow', 'tactic', 'aka', 'BATTERY', 'GATE', 'I', 'figured', 'I', 'would', 'run', 'into', 'a', 'problem', 'like', 'this', 'since', 'it', 's', 'a', 'refurbished', 'one', 'anyway', 'It', 's', 'impossible', 'to', 'know', 'the', 'performance', 'and', 'battery', 'life', 'without', 'activating', 'the

## Cleaning text of stopwords
There are some really basic words that just don't matter. NLTK comes with a list of them for many languages.

In [12]:
tokenized_docs_no_stopwords = []
for doc in tokenized_docs_no_punctuation:
    new_term_vector = []
    for word in doc:
        if not word in stopwords.words('english'):
            new_term_vector.append(word)
    tokenized_docs_no_stopwords.append(new_term_vector)
            
print (tokenized_docs_no_stopwords[1])

['purchased', 'father', 'got', 'one', 'day', 'late', 'big', 'deal', 'phone', 'unlocked', 'item', 'came', 'well', 'packaged', 'listed', 'items', 'plus', 'one', 'glass', 'screen', 'protector', 'one', 'plastic', 'screen', 'protector', 'sure', 'amazon', 'freebie', 'seller', 'freebie', 'able', 'set', 'phone', 'problem', 'great', 'price', 'buy', 'phone', 'somewhere', 'else', 'risk', 'I', 'definitely', 'buy', 'another', 'phone', 'amazon', 'without', 'second', 'thought']


### Stemming and Lemmatizing
If you have taken linguistics, you may be familiar with morphology. This is the belief that words have a root form. If you want to get to the basic term meaning of the word, you can try applying a stemmer or lemmatizer. Here are three very popular methods ready to go right out of the NLTK box. It's up to you to see which one you want to use.

In [13]:
porter = PorterStemmer()
snowball = SnowballStemmer('english')
wordnet = WordNetLemmatizer()

preprocessed_docs = []

for doc in tokenized_docs_no_stopwords:
    final_doc = []
    for word in doc:
        final_doc.append(wordnet.lemmatize(word))
        #final_doc.append(snowball.stem(word))
        #final_doc.append(wordnet.lemmatize(word)) #note that lemmatize() can also takes part of speech as an argument!
    preprocessed_docs.append(final_doc)

print (preprocessed_docs[1])

['purchased', 'father', 'got', 'one', 'day', 'late', 'big', 'deal', 'phone', 'unlocked', 'item', 'came', 'well', 'packaged', 'listed', 'item', 'plus', 'one', 'glass', 'screen', 'protector', 'one', 'plastic', 'screen', 'protector', 'sure', 'amazon', 'freebie', 'seller', 'freebie', 'able', 'set', 'phone', 'problem', 'great', 'price', 'buy', 'phone', 'somewhere', 'else', 'risk', 'I', 'definitely', 'buy', 'another', 'phone', 'amazon', 'without', 'second', 'thought']


In [14]:
pos_tags=[]

for i in preprocessed_docs:
    pos_tags.append(nltk.pos_tag(i)) 

In [15]:
pos_tags[2]

[('So', 'RB'),
 ('I', 'PRP'),
 ('wanted', 'VBD'),
 ('wait', 'NN'),
 ('reviewing', 'VBG'),
 ('case', 'NN'),
 ('It', 'PRP'),
 ('6', 'CD'),
 ('week', 'NN'),
 ('since', 'IN'),
 ('purchase', 'NN'),
 ('phone', 'NN'),
 ('work', 'NN'),
 ('perfect', 'VBP'),
 ('No', 'DT'),
 ('issue', 'NN'),
 ('whatsoever', 'RB'),
 ('And', 'CC'),
 ('brand', 'NN'),
 ('new', 'JJ'),
 ('condition', 'NN'),
 ('I', 'PRP'),
 ('received', 'VBD'),
 ('I', 'PRP'),
 ('ordering', 'VBG'),
 ('another', 'DT'),
 ('soon', 'RB'),
 ('one', 'CD'),
 ('I', 'PRP'),
 ('purchased', 'VBD'),
 ('daughter', 'NN')]

In [16]:
noun_words=[]
verb_words=[]
adj_words=[]
for doc in pos_tags:
    for word_pos in doc :
        if word_pos[1] in ('NN','NNS','NNP','NNPS'):
            noun_words.append(word_pos[0]) 
        elif word_pos[1] in ('VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'):
            verb_words.append(word_pos[0])
        elif word_pos[1] in ('JJR','JJS','JJ'):
            adj_words.append(word_pos[0])
        

In [17]:
print(len(set(adj_words)))

402


In [18]:
noun_dict=dict((i, noun_words.count(i)) for i in noun_words)
noun_dict = sorted(noun_dict.items(), key=operator.itemgetter(1),reverse=True)
print((noun_dict))

[('phone', 380), ('work', 93), ('problem', 65), ('battery', 59), ('time', 58), ('product', 51), ('condition', 40), ('day', 39), ('screen', 37), ('month', 37), ('Phone', 34), ('iPhone', 34), ('money', 32), ('charge', 31), ('issue', 30), ('Great', 30), ('’', 26), ('charger', 24), ('purchase', 24), ('Apple', 24), ('week', 23), ('brand', 23), ('review', 23), ('service', 23), ('price', 20), ('use', 20), ('iphone', 19), ('scratch', 18), ('seller', 18), ('nt', 18), ('hour', 18), ('NOT', 17), ('card', 17), ('daughter', 16), ('camera', 16), ('return', 16), ('Amazon', 14), ('look', 14), ('item', 13), ('thing', 13), ('button', 13), ('Thank', 13), ('apple', 13), ('lot', 12), ('customer', 12), ('store', 12), ('waste', 12), ('SIM', 11), ('DO', 11), ('Battery', 11), ('case', 10), ('sound', 10), ('device', 10), ('minute', 10), ('call', 10), ('turn', 9), ('everything', 9), ('No', 9), ('crack', 9), ('deal', 8), ('Do', 8), ('order', 8), ('get', 8), ('Tried', 8), ('refund', 8), ('delivery', 8), ('gift', 8

In [19]:
verb_dict=dict((i, verb_words.count(i)) for i in verb_words)
verb_dict = sorted(verb_dict.items(), key=operator.itemgetter(1),reverse=True)
print((verb_dict))

[('working', 55), ('nt', 46), ('came', 41), ('buy', 37), ('bought', 35), ('get', 34), ('refurbished', 30), ('got', 28), ('worked', 22), ('love', 22), ('go', 18), ('used', 18), ('received', 17), ('recommend', 17), ('know', 15), ('took', 14), ('stopped', 14), ('’', 14), ('described', 13), ('come', 13), ('work', 12), ('said', 12), ('need', 12), ('charged', 12), ('getting', 12), ('unlocked', 11), ('turned', 11), ('make', 11), ('put', 10), ('told', 10), ('use', 10), ('going', 10), ('keep', 10), ('buying', 10), ('return', 9), ('seems', 9), ('turn', 9), ('pleased', 9), ('started', 9), ('take', 9), ('using', 9), ('Works', 9), ('purchased', 8), ('charging', 8), ('sent', 8), ('expected', 8), ('want', 8), ('see', 8), ('tried', 7), ('ordered', 7), ('give', 7), ('called', 7), ('send', 7), ('thought', 7), ('let', 7), ('advertised', 7), ('like', 7), ('replace', 6), ('broke', 6), ('spent', 6), ('say', 6), ('went', 6), ('locked', 6), ('perfect', 5), ('trying', 5), ('arrived', 5), ('excited', 5), ('cert

In [20]:
adj_dict=dict((i, adj_words.count(i)) for i in adj_words)
adj_dict = sorted(adj_dict.items(), key=operator.itemgetter(1),reverse=True)
print((adj_dict))

[('new', 77), ('good', 68), ('great', 65), ('nt', 30), ('happy', 29), ('bad', 22), ('Good', 15), ('last', 14), ('much', 12), ('perfect', 12), ('fine', 12), ('screen', 11), ('easy', 10), ('first', 10), ('unlocked', 9), ('big', 8), ('old', 8), ('little', 8), ('second', 7), ('disappointed', 7), ('charger', 7), ('original', 7), ('sure', 6), ('able', 6), ('expensive', 6), ('Excellent', 6), ('dead', 6), ('worth', 6), ('free', 6), ('slow', 5), ('several', 5), ('different', 5), ('black', 5), ('call', 5), ('nice', 5), ('refurbished', 5), ('many', 5), ('defective', 5), ('full', 5), ('mic', 4), ('fix', 4), ('hard', 4), ('nervous', 4), ('touch', 4), ('low', 4), ('iPhone', 4), ('open', 4), ('wrong', 4), ('Horrible', 4), ('sound', 4), ('satisfied', 4), ('long', 4), ('generic', 3), ('amazon', 3), ('next', 3), ('due', 3), ('unresponsive', 3), ('hear', 3), ('front', 3), ('extra', 3), ('awesome', 3), ('large', 3), ('previous', 3), ('better', 3), ('normal', 3), ('mobile', 3), ('unable', 3), ('Other', 3),

In [21]:
imp_nouns=[]
imp_verbs=[]
imp_adjs=[]
for k,v in noun_dict:
    if(v>=5):
        imp_nouns.append(k)
for k,v in verb_dict:
    if(v>=5):
        imp_verbs.append(k)
for k,v in adj_dict:
    if(v>=5):
        imp_adjs.append(k)
        

In [22]:
print(len(imp_nouns),len(imp_verbs),len(imp_adjs))

123 81 39


In [23]:
print(imp_nouns)

['phone', 'work', 'problem', 'battery', 'time', 'product', 'condition', 'day', 'screen', 'month', 'Phone', 'iPhone', 'money', 'charge', 'issue', 'Great', '’', 'charger', 'purchase', 'Apple', 'week', 'brand', 'review', 'service', 'price', 'use', 'iphone', 'scratch', 'seller', 'nt', 'hour', 'NOT', 'card', 'daughter', 'camera', 'return', 'Amazon', 'look', 'item', 'thing', 'button', 'Thank', 'apple', 'lot', 'customer', 'store', 'waste', 'SIM', 'DO', 'Battery', 'case', 'sound', 'device', 'minute', 'call', 'turn', 'everything', 'No', 'crack', 'deal', 'Do', 'order', 'get', 'Tried', 'refund', 'delivery', 'gift', 'Product', 'My', 'box', 'quality', 'Looks', 'Works', 'picture', 'touch', 'life', 'star', 'volume', 'buy', 'carrier', 'nothing', 'Would', 'love', 'amazon', 'replacement', 'Siri', 'hardware', 'Just', 'let', 'Came', 'speaker', 'Bought', 'Got', 'perfect', 'Iphone', 'Was', 'today', 'BUY', 'something', 'Excellent', 'country', 'glass', 'shutter', 'people', 'shape', 'IT', 'company', 'warranty'

In [23]:
#data_samples=preprocessed_docs

In [24]:
#data_samples=data['Review_Text'].tolist()

In [24]:
# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2,
                                   stop_words='english')

Extracting tf-idf features for NMF...


In [25]:
lemmatised_docs=[]
for i in preprocessed_docs:
    str=' '.join(i)
    lemmatised_docs.append(str)
    

In [26]:
lemmatised_docs[5]

'It work clean Professional packaging arrived fastDownside battery must replaced wo nt even last half day idle Good thing least nt waterproof phone I easily buy battery replacement kit le 30 Amazon It really would great put new battery wo nt suffer performance due shit iOS get new phone old phone slow tactic aka BATTERY GATE I figured I would run problem like since refurbished one anyway It impossible know performance battery life without activating phone I bet lot people problem would actually return activated But hey I paying thousand new phone I bought iPhone knowing ongoing battery gate controversy I suck wo nt even bother sending complaintBut I chose give 3 star since price nt bad seems easy enough replace battery nt cost much buy battery kit I nt like okay'

In [27]:
print(type(data_samples[1]),type(lemmatised_docs[1]))

<class 'str'> <class 'str'>


In [28]:
tfidf = tfidf_vectorizer.fit_transform(lemmatised_docs)

In [29]:
tfidf.shape

(570, 621)

In [30]:
tfidf_features=tfidf_vectorizer.get_feature_names()
len(tfidf_features)

621

In [31]:
tfidf_features[:20]

['10',
 '100',
 '20',
 '30',
 '300',
 '45',
 '50',
 '500',
 '80',
 '90',
 '98',
 'able',
 'absolutely',
 'account',
 'activate',
 'activated',
 'activating',
 'activation',
 'actually',
 'add']

In [32]:
print(tfidf.shape)
# shape of the tfidf is 26,105 means 26 docs(reviews) and 105 features(words) with max_df=0.90, min_df=2

(570, 621)


In [33]:
tfidf.getrow(0)

<1x621 sparse matrix of type '<class 'numpy.float64'>'
	with 18 stored elements in Compressed Sparse Row format>

https://buhrmann.github.io/tfidf-analysis.html

In [34]:
def top_tfidf_feats(row, features,top_n):
    #top_n=25
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

In [35]:
def top_feats_in_doc(Xtr, features, row_id,top_n):
    #top_n=25
    ''' Top tfidf features in specific document (matrix row) '''
    row = np.squeeze(Xtr[row_id].toarray())
    return top_tfidf_feats(row, features,top_n)

In [36]:
def top_mean_feats(Xtr, features,top_n):
    grp_ids=None
    min_tfidf=0.1
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in grp_ids. '''
    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_means, features,top_n)

In [37]:
def top_feats_by_class(Xtr, y, features,top_n):
    min_tfidf=0.1
    
    ''' Return a list of dfs, where each df holds top_n features and their mean tfidf value
        calculated across documents with the same class label. '''
    dfs = []
    labels = np.unique(y)
    for label in labels:
        ids = np.where(y==label)
        feats_df = top_mean_feats(Xtr, features)
        feats_df.label = label
        dfs.append(feats_df)
    return dfs

In [38]:
top_feats_in_doc(tfidf,tfidf_features,5,5)

,feature,tfidf
0,battery,0.478978
1,nt,0.367311
2,wo,0.274682
3,performance,0.231192
4,new,0.175245


In [39]:
top_mean_feats(tfidf,tfidf_features,50)

,feature,tfidf
0,phone,0.083734
1,great,0.063209
2,good,0.060543
3,work,0.039250
4,nt,0.032901
5,product,0.030081
6,new,0.030068
7,love,0.029118
8,problem,0.028558
9,battery,0.028178


In [40]:
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                               stop_words='english')

Extracting tf features for LDA...


In [41]:
tf = tf_vectorizer.fit_transform(lemmatised_docs)

In [42]:
print(tf_vectorizer.vocabulary)

None


In [43]:
tf_features=tf_vectorizer.get_feature_names()
len(tf_features)

621

In [44]:
tf_vectorizer.get_feature_names()[:20]

['10',
 '100',
 '20',
 '30',
 '300',
 '45',
 '50',
 '500',
 '80',
 '90',
 '98',
 'able',
 'absolutely',
 'account',
 'activate',
 'activated',
 'activating',
 'activation',
 'actually',
 'add']

In [45]:
print(tf.shape)

(570, 621)


In [46]:
tf.data

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [49]:
# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (1000, 1000))
nmf = NMF( random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=1000 and n_features=1000...


In [50]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [51]:
print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, 50)


Topics in NMF model (Frobenius norm):
Topic #0: zero finally freeze free flawlessly flat fix fit fine feel factory faulty father fast far family fallen fake freezing friend frustrated fully good gold going glitchy glitch glass glad given gift getting generic gave garbage functioning function fairly fact gotten drain easy easily earphone dying dy durable drop doubt facing
Topic #1: zero finally freeze free flawlessly flat fix fit fine feel factory faulty father fast far family fallen fake freezing friend frustrated fully good gold going glitchy glitch glass glad given gift getting generic gave garbage functioning function fairly fact gotten drain easy easily earphone dying dy durable drop doubt facing
Topic #2: zero finally freeze free flawlessly flat fix fit fine feel factory faulty father fast far family fallen fake freezing friend frustrated fully good gold going glitchy glitch glass glad given gift getting generic gave garbage functioning function fairly fact gotten drain easy easi

In [52]:
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (1000, 1000))
nmf = NMF( random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)

Fitting the NMF model (generalized Kullback-Leibler divergence) with tf-idf features, n_samples=1000 and n_features=1000...


In [53]:
print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, 10)


Topics in NMF model (generalized Kullback-Leibler divergence):
Topic #0: unlocked phone zero fine freeze free flawlessly flat fix fit
Topic #1: nt zero finally freeze free flawlessly flat fix fit fine
Topic #2: zero finally freeze free flawlessly flat fix fit fine feel
Topic #3: old phone finally freeze free flawlessly flat fix fit fine
Topic #4: need password unlock freezing free flawlessly flat fix fit fine
Topic #5: zero finally freeze free flawlessly flat fix fit fine feel
Topic #6: time attention level paying durable pas lesson poor read fit
Topic #7: broken zero fine freezing freeze free flawlessly flat fix fit
Topic #8: zero finally freeze free flawlessly flat fix fit fine feel
Topic #9: load friend minute lasted solve night left called 20 turned
Topic #10: zero finally freeze free flawlessly flat fix fit fine feel
Topic #11: real iphone zero finally free flawlessly flat fix fit fine
Topic #12: expected feel free flawlessly flat fix fit fine finally zero
Topic #13: glitchy rece

In [54]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (1000, 1000))
lda = LatentDirichletAllocation(max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

Fitting LDA models with tf features, n_samples=1000 and n_features=1000...


In [55]:
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [56]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, 20)


Topics in LDA model:
Topic #0: thank monday waiting randomly happy happening turning known ordered fully new sound bad refurbished problem complaint sure just let iphone
Topic #1: loved yes awesome thanks suppose satisfied daughter phone guy described mom network works opened flawlessly sprint samsung buy iphones happier
Topic #2: worked store review battery new apple said gold hold yesterday instead needed sold buying perfectly issue charge id set excellent
Topic #3: issue phone speaker called minute battery load charged blue hear worked 20 amazing friend completely bought mess problem work person
Topic #4: terrible supplier garbage dollar zero product data reason time stuck charge phone sell far like star minute suck gave damaged
Topic #5: phone nt iphone great work good new battery product problem time buy working screen like love came apple condition refurbished
Topic #6: broke crack corner day phone caused ok unless pretty screen notice cracked difficult received receiving flat h

In [65]:
# Create training and test sets
train, test = train_test_split(data['Review_Text'],test_size=0.2,random_state=1)

In [66]:
test.shape

(114,)

In [68]:
vect = CountVectorizer().fit(train)
vect.get_feature_names()[::50]

['00',
 'act',
 'apple',
 'behind',
 'business',
 'charger',
 'considerably',
 'dead',
 'doesn',
 'evade',
 'faster',
 'freezes',
 'gonna',
 'hence',
 'info',
 'keys',
 'little',
 'marks',
 'move',
 'nytimes',
 'packaged',
 'popped',
 'protesters',
 'refund',
 'safest',
 'seven',
 'someone',
 'sucked',
 'them',
 'twisted',
 've',
 'wht']

In [69]:
len(vect.get_feature_names())

1594

In [70]:
# transform the documents in the training data to a document-term matrix
train_vectorized = vect.transform(train)

train_vectorized

<456x1594 sparse matrix of type '<class 'numpy.int64'>'
	with 8296 stored elements in Compressed Sparse Row format>

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Fit the TfidfVectorizer to the training data specifiying a minimum document frequency of 5
tfidfvect = TfidfVectorizer(min_df=10).fit(train)
len(tfidfvect.get_feature_names())

165

In [72]:
tfidfvect.get_feature_names()[100:120]

['perfectly',
 'phone',
 'phones',
 'price',
 'problem',
 'problems',
 'product',
 'properly',
 'purchase',
 'really',
 'received',
 'recommend',
 'refurbished',
 'return',
 'reviews',
 'scratches',
 'screen',
 'seller',
 'send',
 'service']